# Post-MVP Modeling

The purpose of this notebook is to improve the final model aka "Minimum Viable Product" that predicted with an F1 of 0.3955.

## Method 1: Linear SVM with Count Vectorization

## Method 2: Offensive Language as Stop Words